# Lab 01: Phishing Email Classifier

Build a machine learning classifier to detect phishing emails.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab01_phishing_classifier.ipynb)

## Learning Objectives
- Text preprocessing and feature extraction
- TF-IDF vectorization
- Random Forest classification
- Model evaluation (precision, recall, F1)

In [ ]:
# Install dependencies (uncomment for Colab)
# !pip install scikit-learn pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

## 1. Load and Explore Data

In [ ]:
# Sample phishing email dataset
data = {
    'text': [
        'Urgent: Your account has been compromised. Click here to verify your identity immediately.',
        'Dear customer, your package is waiting. Track it here: bit.ly/suspicious',
        'You have won $1,000,000! Claim your prize now by clicking this link.',
        'Hi team, please review the attached quarterly report before the meeting.',
        'Meeting reminder: Project sync at 3pm today in Conference Room A.',
        'Your invoice #12345 is attached. Please remit payment within 30 days.',
        'URGENT: Your bank account will be suspended. Verify now!',
        'Security Alert: Unusual login detected. Reset password immediately.',
        'Congratulations! You\'ve been selected for a special offer. Act now!',
        'Please find attached the meeting notes from yesterday.',
        'Your order has shipped. Track your package with the link below.',
        'Reminder: Team lunch tomorrow at noon.',
        'ACTION REQUIRED: Update your payment information to avoid service interruption.',
        'Your subscription is expiring. Renew now to save 50%!',
        'Weekly newsletter: Industry updates and insights.',
    ],
    'label': [1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0]  # 1 = phishing, 0 = legitimate
}

df = pd.DataFrame(data)
print(f"Dataset shape: {df.shape}")
print(f"\nClass distribution:")
print(df['label'].value_counts())

In [ ]:
# Visualize class distribution
fig, ax = plt.subplots(figsize=(8, 5))
colors = ['#2ecc71', '#e74c3c']
df['label'].value_counts().plot(kind='bar', ax=ax, color=colors)
ax.set_xticklabels(['Legitimate', 'Phishing'], rotation=0)
ax.set_xlabel('Email Type')
ax.set_ylabel('Count')
ax.set_title('Email Classification Distribution')
plt.tight_layout()
plt.show()

## 2. Feature Extraction with TF-IDF

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.3, random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(
    max_features=1000,
    stop_words='english',
    ngram_range=(1, 2)  # Unigrams and bigrams
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Feature matrix shape: {X_train_tfidf.shape}")
print(f"\nTop 10 features:")
feature_names = vectorizer.get_feature_names_out()
print(feature_names[:10])

## 3. Train Random Forest Classifier

In [ ]:
# Train model
clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

clf.fit(X_train_tfidf, y_train)
print("Model trained successfully!")

## 4. Evaluate Model Performance

In [ ]:
# Predictions
y_pred = clf.predict(X_test_tfidf)
y_prob = clf.predict_proba(X_test_tfidf)[:, 1]

# Classification report
print("Classification Report:")
print("=" * 50)
print(classification_report(y_test, y_pred, target_names=['Legitimate', 'Phishing']))

In [ ]:
# Confusion Matrix
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix Heatmap
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Legitimate', 'Phishing'],
            yticklabels=['Legitimate', 'Phishing'])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')
axes[0].set_title('Confusion Matrix')

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

axes[1].plot(fpr, tpr, color='#3498db', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
axes[1].plot([0, 1], [0, 1], 'k--', lw=1)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve')
axes[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

## 5. Feature Importance Analysis

In [ ]:
# Get feature importances
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1][:20]

# Plot top features
fig, ax = plt.subplots(figsize=(10, 6))
top_features = [feature_names[i] for i in indices]
top_importances = importances[indices]

colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(top_features)))
ax.barh(range(len(top_features)), top_importances, color=colors)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features)
ax.invert_yaxis()
ax.set_xlabel('Feature Importance')
ax.set_title('Top 20 Phishing Indicators')
plt.tight_layout()
plt.show()

## 6. Test with New Emails

In [ ]:
def classify_email(email_text):
    """Classify a single email as phishing or legitimate."""
    email_tfidf = vectorizer.transform([email_text])
    prediction = clf.predict(email_tfidf)[0]
    probability = clf.predict_proba(email_tfidf)[0]
    
    result = "PHISHING" if prediction == 1 else "LEGITIMATE"
    confidence = probability[prediction] * 100
    
    return result, confidence

# Test emails
test_emails = [
    "URGENT: Your account will be closed. Click here immediately!",
    "Hi, let's catch up over coffee next week. How's Tuesday?",
    "You've won a free iPhone! Claim now before it expires!",
]

print("Email Classification Results:")
print("=" * 60)
for email in test_emails:
    result, confidence = classify_email(email)
    icon = "🚨" if result == "PHISHING" else "✅"
    print(f"\n{icon} {result} ({confidence:.1f}% confidence)")
    print(f"   Email: {email[:50]}...")

## Summary

In this lab, we built a phishing email classifier using:
- **TF-IDF vectorization** to convert email text to numerical features
- **Random Forest classifier** for robust classification
- **Evaluation metrics** including precision, recall, F1, and ROC-AUC

### Key Phishing Indicators Identified:
- Urgency words ("urgent", "immediately", "act now")
- Financial incentives ("won", "prize", "free")
- Security threats ("compromised", "suspended", "verify")

### Next Steps:
1. Add more training data
2. Try deep learning models (BERT, RoBERTa)
3. Add header analysis features
4. Integrate with email gateway